<a href="https://colab.research.google.com/github/heytanay/deep-boltzmann-model/blob/master/Deep_Restricted_Boltzmann_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Get the Dataset Files

In [0]:
from google.colab import files
files.upload()

In [0]:
from google.colab import files
files.upload()

#2. Install PyTorch-GPU in Colab (if not previously installed)

In [0]:
!pip3 install torch torchvision

#3. Data Pre-processing

##Get Basic Library imports done

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

##Unzip and Import the Dataset

In [0]:
!unzip "./ml-1m.zip" && unzip "./ml-100k.zip" 

In [0]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [56]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [58]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [0]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [60]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


##Prepare the Training Set and Test Set

In [0]:
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t')

In [62]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [0]:
testing_set = pd.read_csv('ml-100k/u1.test',delimiter='\t')

In [64]:
testing_set.head()

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


##Convert the Training and Testing set into Arrays from DataFrame

In [0]:
trainS = np.array(training_set, dtype='int')
testS = np.array(testing_set, dtype='int')

##Get the Total number of Users and Movies

In [0]:
nb_users = int(max(max(trainS[:, 0]),max(testS[:, 0])))
nb_movies = int(max(max(trainS[:, 1]),max(testS[:, 1])))

##Convert the Users into Lines and Movies into Columns

In [0]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users+1):
    
    # Get the Id of movies for the present user in the loop (id_users)
    id_movies = data[:, 1][data[:, 0] == id_users]
    
    # Get the Id of ratings for the present user in the loop (id_users)
    id_ratings = data[:, 2][data[:, 0] == id_users]
    
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    
    new_data.append(list(ratings))
  return new_data

In [0]:
trainSConvert = convert(trainS)
testSConvert =  convert(testS)

#3. Convert the Data into Torch Tensors

In [0]:
train_set = torch.FloatTensor(trainSConvert)
test_set = torch.FloatTensor(testSConvert)

##Convert all the ratings in Binary Ratings (1: Liked) or (0: Not Liked)

In [0]:
train_set[train_set == 0] = -1
train_set[train_set == 1] = 0
train_set[train_set == 2] = 0
train_set[train_set >= 2] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 2] = 1

#4. Create the Architecture of the Neural Network

And for the Architecture, I am building a Probabilistic Graphical Model (a boltzmann machine is a Probabilistic Graphical Model)

In [0]:
class RBM():
  def __init__(self, num_visible, num_hidden):
    
    self.W = torch.randn(num_hidden, num_visible)
    self.a = torch.randn(1, num_hidden)
    self.b = torch.randn(1, num_visible)
    
  def sample_h(self, x):
    Wx = torch.mm(x, self.W.t())
    activation = Wx + self.a.expand_as(Wx)
    
    prob_h_given_v = torch.sigmoid(activation)
    
    return prob_h_given_v, torch.bernoulli(prob_h_given_v)
  
  